In [1]:
import os
os.chdir('/mnt/jw01-aruk-home01/projects/psa_functional_genomics/RA_challenge/RA2_alpine_lads')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras 
import os
import tensorflow as tf
import tensorflow_addons as tfa
from utils.config import Config
import dataset.train_dataset as dtd
import PIL
import PIL.ImageOps
from tensorflow.keras.utils import plot_model
test_config = Config()
from tensorflow.keras.models import load_model
from model import NASnet_multioutput
import itertools
import seaborn as sns

In [3]:
from dataset.train_dataset import train_dataset

dataset = train_dataset(test_config)

hands_dataset, feet_dataset, hands_dataset_val, feet_dataset_val = dataset.initialize_pipeline()

WARNING - Missing permissions to create directory for caching!
WARNING - Missing permissions to create directory for caching!


In [4]:
model = NASnet_multioutput.create_NASnet_multioutupt(test_config)
model.load_weights("weights/NIH_chest_NASnet_model_275_step2")

In [5]:
new_model = keras.models.Sequential()
new_model.add(model.layers[0])
new_model.add(model.layers[1])
new_model.add(model.layers[2])
new_model.add(keras.layers.Dense(13,activation="linear",name="State"))
for layer in new_model.layers[:-1]:
    layer.trainable = False
new_model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mae"])

In [6]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 16, 12, 1056)      4269140   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1056)              0         
_________________________________________________________________
State (Dense)                (None, 13)                13741     
Total params: 4,282,881
Trainable params: 13,741
Non-trainable params: 4,269,140
_________________________________________________________________


In [ ]:
history = new_model.fit(
    feet_dataset,validation_data=feet_dataset_val,epochs=10, steps_per_epoch=50,
    validation_steps=2
)

Train for 50 steps, validate for 2 steps
Epoch 1/10
50/50 [==============================] - 88s 2s/step - loss: 4.2286 - mae: 0.6633 - val_loss: 8.4370 - val_mae: 0.8074
Epoch 2/10
50/50 [==============================] - 70s 1s/step - loss: 3.6354 - mae: 0.6423 - val_loss: 8.6933 - val_mae: 0.7892
Epoch 3/10
50/50 [==============================] - 70s 1s/step - loss: 4.0774 - mae: 0.6827 - val_loss: 8.7951 - val_mae: 0.8151
Epoch 4/10
50/50 [==============================] - 72s 1s/step - loss: 3.9636 - mae: 0.6619 - val_loss: 8.9545 - val_mae: 0.8505
Epoch 5/10
34/50 [===================>..........] - ETA: 21s - loss: 3.9929 - mae: 0.6719

In [ ]:
for image, label in feet_dataset_val.take(5):
    N = image.shape[0]
    D = label.shape[1]
    
    error = np.zeros((N, D))
    
    for n in range(N):
        y_pred = model.predict(np.array(image[n]).reshape(1,test_config.img_height,test_config.img_width,1))
        y = label[n]
        
        
        E[n, :] = np.square(y_pred - y)
        
    rmse = np.sqrt(np.mean(E, axis = 0))
        
    print("Batch RMSE: ", rmse)